# Redes Neurais Artificiais 2020.1 -- Projeto Prático 3.4

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você dseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

## Importações
Nesta seção, foi feita:
* Importações de bibliotecas.
* Importação do dataset forest cover type `covtype.csv`.
* Carregamento das top 6 arquiteturas obtidas no notebook anterior `Projeto 3.3`.  
* Remoção dos atributos categóricos do dataset

In [151]:
## Reservado para a importação de bibliotecas

import pandas as pd
from prettytable import PrettyTable  
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from joblib import dump, load

### Atributos do dataset
é importante citar que como os maiores scores obtidos no notebook anterior foram sem o uso dos atributos categóricos, este também nao usará tais atributos.

Nesta célula foi feito o carregamento do dataset e a remoção dos atributos categóricos.

In [199]:
# Leitura do dataset covtype.csv

# df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/covtype.csv', sep=',')  # caso use google colab
df = pd.read_csv('./covtype.csv')                                                   # caso faça localmente pelo jupyter

atributosCategoricos = []
for i in range(40):                                                                 # loop para preencher um vetor com os atributos categoricos
  if i <= 3:
    atributosCategoricos.append("Wilderness_Area"+str(i+1))
  atributosCategoricos.append("Soil_Type"+str(i+1))

df = df.drop(columns=atributosCategoricos)    

x_preditor = df.drop(columns=["Cover_Type"]) 
y_alvo = df["Cover_Type"]  

In [153]:
# criacao do vetor com as 6 melhores configuracoes da arquitetura propria
top6_arquiteturas = []

# preenchimento do vetor com as 3 melhores configuracoes da arquitetura propria
top6_arquiteturas.append({                                                          
    "camadasOcultas": 1,"funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 15, "disposicaoNeuronios": 15, "acuracia": 0.73191,
    "fscore":0.54205
})
top6_arquiteturas.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "tanh", "hiperparametro": "sgd",
    "epocas": 200, "neuronios": 13, "disposicaoNeuronios": 13, "acuracia": 0.72023,
    "fscore":0.47931
})
top6_arquiteturas.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "tanh", "hiperparametro": "sgd",
    "epocas": 200, "neuronios": 11, "disposicaoNeuronios": 11, "acuracia": 0.71824,
    "fscore":0.47116
})

# preenchimento do vetor com as 3 melhores configuracoes da arquitetura priramide
top6_arquiteturas.append({                                                          
    "camadasOcultas": 1,"funcaoAtivacao": "tanh", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 25, "disposicaoNeuronios": 25,  "acuracia": 0.74959,
    "fscore":0.60096
})
top6_arquiteturas.append({                                                          
    "camadasOcultas": 1, "funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 25, "disposicaoNeuronios": 25, "acuracia": 0.74573,
    "fscore":0.58238
})
top6_arquiteturas.append({                                                          
    "camadasOcultas": 2, "funcaoAtivacao": "logistic", "hiperparametro": "adam",
    "epocas": 200, "neuronios": 25, "disposicaoNeuronios": (22, 3), "acuracia": 0.74294,
    "fscore":0.53948
})

### Demonstração das arquiteturas
Nesta seção foi feita o demonstração, em forma de tabela, das top 6 configurações que serão usadas nesse projeto.

In [200]:
# Exibicao das configuracoes top 6

table = PrettyTable(["Camadas","Função",                                          # Criacao da tabela
                      "Hiperparametro", "Épocas", 
                      "Neurônios", "Neurônios por camada", 
                     "Acurácia", "F-Score", "Categóricos"])        

for i in top6_arquiteturas:                                                       # loop para preencher a tabela com os dados
  table.add_row([i["camadasOcultas"], i["funcaoAtivacao"], i["hiperparametro"],
                i["epocas"], i["neuronios"], i["disposicaoNeuronios"],
                i["acuracia"], i["fscore"], "Não"])

# print da tabela com as configuracoes geradas
print(table)

+---------+----------+----------------+--------+-----------+----------------------+----------+---------+-------------+
| Camadas |  Função  | Hiperparametro | Épocas | Neurônios | Neurônios por camada | Acurácia | F-Score | Categóricos |
+---------+----------+----------------+--------+-----------+----------------------+----------+---------+-------------+
|    1    | logistic |      adam      |  200   |     15    |          15          | 0.73191  | 0.54205 |     Não     |
|    1    |   tanh   |      sgd       |  200   |     13    |          13          | 0.72023  | 0.47931 |     Não     |
|    1    |   tanh   |      sgd       |  200   |     11    |          11          | 0.71824  | 0.47116 |     Não     |
|    1    |   tanh   |      adam      |  200   |     25    |          25          | 0.74959  | 0.60096 |     Não     |
|    1    | logistic |      adam      |  200   |     25    |          25          | 0.74573  | 0.58238 |     Não     |
|    2    | logistic |      adam      |  200   |

## Busca em grade usando validação cruzada 5-Fold

Nesta seção foi feita:
* Criação das listas que serão usadas na validação cruzada.
* Uso da biblioteca `GridSearchCV` para validação cruzada.
* Treino das arquiteturas obtidas a partir do produto carteziano das listas.

> É importante ressaltar que por conta de recursos computacionais limitados, foi usado listas de tamanho 2, resultando assim em 16 configurações diferentes a partir do produto cartesiano. E também que a biblioteca `GridSearchCV` usa como padrão um k-fold de 5 por isso o mesmo não foi especificado na criação da busca.

In [188]:
# criacao das configuracoes para busca em grade

GridSearchList = []                                                               # criacao da lista para armazenar as 6 configuracoes

scoring = {                                                                       # criacao do dicionario para o score f-score
    'f1_score': make_scorer(f1_score, average="macro")
}

SolverList = ["sgd", "adam"]                                                      # lista com os parametros de solver
BatchSizeList = [350000]                                                          # lista com o parametros de batch size
LearningRateList = [0.01, 0.02]                                                   # lista com os valores de taxa incial de aprendizagem
PatienceList = [40, 50]                                                           # lista com os valores de paciencia
Epochs = [500, 600]                                                               # lista com os valores de epocas

tuned_parameters ={                                                               # criacao do dicionario com os parametros para a busca em grade
    "solver": SolverList,
    "batch_size": BatchSizeList,
    "learning_rate_init": LearningRateList,
    "n_iter_no_change": PatienceList,
    "max_iter": Epochs
} 
cont = 0
for i in top6_arquiteturas:                                                       # loop para criacao, busca em grade e treino das top6 configuracoes
    cont = cont + 1
    print("Executando configuração {} de 6".format(cont))
    classifier = MLPClassifier(hidden_layer_sizes=(i["disposicaoNeuronios"]),     # criacao do modelo          
                            activation=i["funcaoAtivacao"], 
                            solver=i["hiperparametro"], 
                            max_iter=i["epocas"], 
                            verbose=False)
    
    clf = GridSearchCV(classifier, tuned_parameters, scoring=scoring, refit="f1_score") # criacao da busca em grade
    clf.fit(x_preditor, y_alvo)                                                   # treino usando os valores da busca em grade
    
    GridSearchList.append(clf)                                                    # insecao na lista para posterior avaliacao

## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

In [193]:
bestScore = 0.0
bestEstimator = 0

for idx, val in enumerate(GridSearchList):
    if bestScore < val.best_score_:
        bestScore = val.best_score_
        bestEstimator = idx

paramsTable = PrettyTable(["Camadas","Função",                              # Criacao da tabela com os parametros
                           "Hiperparametro", "Épocas", 
                           "Neurônios", "Neurônios por camada", 
                           "Categóricos"])
paramsTable.add_row([top6_arquiteturas[bestEstimator]["camadasOcultas"], 
               top6_arquiteturas[bestEstimator]["funcaoAtivacao"], 
               top6_arquiteturas[bestEstimator]["hiperparametro"], 
               top6_arquiteturas[bestEstimator]["epocas"], 
               top6_arquiteturas[bestEstimator]["neuronios"], 
               top6_arquiteturas[bestEstimator]["disposicaoNeuronios"], "Não"])
print("\n------------------------------------------- Parâmetros ------------------------------------------")
print(paramsTable)

hiperParamsTable = PrettyTable(["Batch size","Learning rate init",          # Criacao da tabela com hiperparametros otimizados
                                "Épocas", "Paciência", 
                                "Solver"])
hiperParamsTable.add_row([GridSearchList[bestEstimator].best_params_["batch_size"], 
               GridSearchList[bestEstimator].best_params_["learning_rate_init"], 
               GridSearchList[bestEstimator].best_params_["max_iter"], 
               GridSearchList[bestEstimator].best_params_["n_iter_no_change"], 
               GridSearchList[bestEstimator].best_params_["solver"]])
print("\n------------------- Hiperparâmetros Otimizados ------------------")
print(hiperParamsTable)

kFoldsTable = PrettyTable(["K-Folds", "Config 1","Config 2",               # Criacao da tabela com os resultados k-folds
                           "Config 3", "Config 4", "Config 5",
                          "Config 6", "Config 7", "Config 8"])
for i in range(5):
    kFoldsTable.add_row([i+1, round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][0], 5), 
                   round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][1], 5), 
                   round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][2], 5), 
                   round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][3], 5),
                   round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][4], 5),
                   round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][5], 5),
                   round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][6], 5),
                   round(GridSearchList[bestEstimator].cv_results_["split"+str(i)+"_test_f1_score"][7], 5)])
print("\n-------------------------------------------- K-Folds ----------------------------------------------")
print(kFoldsTable)


------------------------------------------- Parâmetros ------------------------------------------
+---------+----------+----------------+--------+-----------+----------------------+-------------+
| Camadas |  Função  | Hiperparametro | Épocas | Neurônios | Neurônios por camada | Categóricos |
+---------+----------+----------------+--------+-----------+----------------------+-------------+
|    1    | logistic |      adam      |  200   |     25    |          25          |     Não     |
+---------+----------+----------------+--------+-----------+----------------------+-------------+

------------------- Hiperparâmetros Otimizados ------------------
+------------+--------------------+--------+-----------+--------+
| Batch size | Learning rate init | Épocas | Paciência | Solver |
+------------+--------------------+--------+-----------+--------+
|   350000   |        0.01        |  500   |     40    |  adam  |
+------------+--------------------+--------+-----------+--------+

-------------

## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html
2. Treine o modelo com todos os dados
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas

In [201]:
# Criacao do modelo otimizado
optimizedClassifier = MLPClassifier(hidden_layer_sizes=top6_arquiteturas[bestEstimator]["disposicaoNeuronios"],
                        activation=top6_arquiteturas[bestEstimator]["funcaoAtivacao"], 
                        solver=GridSearchList[bestEstimator].best_params_["solver"], 
                        max_iter=GridSearchList[bestEstimator].best_params_["max_iter"],
                        batch_size=GridSearchList[bestEstimator].best_params_["batch_size"],
                        learning_rate_init=GridSearchList[bestEstimator].best_params_["learning_rate_init"],
                        n_iter_no_change=GridSearchList[bestEstimator].best_params_["n_iter_no_change"],
                        verbose=False)

# treino do modelo criado a cima
optimizedClassifier.fit(x_preditor, y_alvo)

In [197]:
# Persistencia do modelo em disco
dump(optimizedClassifier, 'MLPClassifier.joblib') 

['MLPClassifier.joblib']

In [198]:
# Recuperação do modelo
model = load('MLPClassifier.joblib') 

# Prevendo para todos os exemplos do dataset sem atributos categoricos
y_test = model.predict(x_preditor)

# matriz de confusao para os atributos do dataset
print(confusion_matrix(y_test, y_alvo))

[[136260  45738      0      0    159     12  19672]
 [ 75498 231128   8215    154   9203   6215    838]
 [    82   6435  27539   2593    131  11140      0]
 [     0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0]]
